In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tabm import TabM  # Ensure TabM is installed and imported  

# ------------------------------ Config ------------------------------

class Config:
    TRAIN_PATH = 'train.csv'
    TEST_PATH = 'test.csv'
    TARGET_COLS = [f'BlendProperty{i}' for i in range(1, 11)]
    TEST_SIZE = 0.1
    BATCH_SIZE = 32
    LR = 1e-3
    EPOCHS = 300
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    K_ENSEMBLE = 64
    RANDOM_STATE = 42

# ------------------------------ Dataset ------------------------------

class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

# ------------------------------ Trainer ------------------------------

class Trainer:
    def __init__(self, config):
        self.config = config
        self.device = config.DEVICE
        self.criterion_mse = nn.MSELoss()
        self.load_and_prepare_data()
        self.create_model()

    def load_and_prepare_data(self):
        train_df = pd.read_csv(self.config.TRAIN_PATH)
        test_df = pd.read_csv(self.config.TEST_PATH)
        train_df = self.feature_engineering(train_df, fit=True)
        test_df = self.feature_engineering(test_df, fit=False)

        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            train_df.drop(columns=self.target_cols).values,
            train_df[self.target_cols].values,
            test_size=self.config.TEST_SIZE,
            random_state=self.config.RANDOM_STATE
        )

        self.X_test = test_df.drop(columns=self.target_cols, errors='ignore').values
        self.test_ids = test_df['ID'].values if 'ID' in test_df.columns else np.arange(len(test_df))

        self.num_scaler = RobustScaler()
        self.y_scaler = RobustScaler()

        self.X_train = self.num_scaler.fit_transform(self.X_train)
        self.X_val = self.num_scaler.transform(self.X_val)
        self.X_test = self.num_scaler.transform(self.X_test)

        self.y_train = self.y_scaler.fit_transform(self.y_train)
        self.y_val = self.y_scaler.transform(self.y_val)

        self.train_loader = DataLoader(TabularDataset(self.X_train, self.y_train), batch_size=self.config.BATCH_SIZE, shuffle=True)
        self.val_loader = DataLoader(TabularDataset(self.X_val, self.y_val), batch_size=self.config.BATCH_SIZE, shuffle=False)
        self.test_loader = DataLoader(TabularDataset(self.X_test), batch_size=self.config.BATCH_SIZE, shuffle=False)

    def feature_engineering(self, df, fit=False):
        df = df.copy()
        
        # Ensure all target columns exist — fill missing with zeros
        for col in Config.TARGET_COLS:
            if col not in df.columns:
                df[col] = 0

        # Drop identifier columns if present (e.g. 'ID', 'BlendID')
        id_cols = ['ID', 'BlendID']
        df.drop(columns=[col for col in id_cols if col in df.columns], inplace=True)

        if fit:
            # Determine feature columns during training
            self.feature_cols = [col for col in df.columns if col not in Config.TARGET_COLS]
            self.target_cols = Config.TARGET_COLS

        # Ensure consistent column ordering
        df = df[self.feature_cols + self.target_cols]

        return df


    def create_model(self):
        self.model = TabM.make(
            n_num_features=self.train_loader.dataset.X.shape[1],
            cat_cardinalities=None,
            d_out=len(self.target_cols),
            k=self.config.K_ENSEMBLE
        ).to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.config.LR)

    def train_epoch(self, epoch):
        self.model.train()
        total_loss = 0
        for X, y in tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.config.EPOCHS}"):
            X, y = X.to(self.device), y.to(self.device)
            preds = self.model(X)
            preds_median = torch.median(preds, dim=1)[0]
            loss = self.criterion_mse(preds_median, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(self.train_loader)

    def validate(self):
        self.model.eval()
        total_loss = 0
        all_preds, all_targets = [], []

        with torch.no_grad():
            for X, y in self.val_loader:
                X, y = X.to(self.device), y.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                loss = self.criterion_mse(preds_median, y)
                total_loss += loss.item()

                all_preds.append(preds_median.cpu().numpy())
                all_targets.append(y.cpu().numpy())

        preds_all = np.vstack(all_preds)
        targets_all = np.vstack(all_targets)

        preds_all_orig = self.y_scaler.inverse_transform(preds_all)
        targets_all_orig = self.y_scaler.inverse_transform(targets_all)
        mape = mean_absolute_percentage_error(targets_all_orig, preds_all_orig) * 100

        return total_loss / len(self.val_loader), mape

    def train(self):
        for epoch in range(self.config.EPOCHS):
            train_loss = self.train_epoch(epoch)
            val_loss, val_mape = self.validate()
            print(f"Epoch {epoch+1}/{self.config.EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAPE: {val_mape:.2f}%")

    def predict(self):
        self.model.eval()
        preds_all = []

        with torch.no_grad():
            for X in self.test_loader:
                X = X.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                preds_all.append(preds_median.cpu().numpy())

        preds_all = np.vstack(preds_all)
        preds_orig = self.y_scaler.inverse_transform(preds_all)

        submission = pd.DataFrame(preds_orig, columns=self.target_cols)
        submission.insert(0, 'ID', self.test_ids)
        submission.to_csv('submission3(1).csv', index=False)
        print("✅ Submission saved to submission.csv")

# ------------------------------ Run ------------------------------

if __name__ == '__main__':
    config = Config()
    trainer = Trainer(config)
    trainer.train()
    trainer.predict()


Epoch 1/300: 100%|██████████| 57/57 [00:05<00:00,  9.67it/s]


Epoch 1/300 | Train Loss: 0.4795 | Val Loss: 0.2699 | Val MAPE: 1928.50%


Epoch 2/300: 100%|██████████| 57/57 [00:05<00:00, 10.57it/s]


Epoch 2/300 | Train Loss: 0.2067 | Val Loss: 0.1555 | Val MAPE: 552.20%


Epoch 3/300: 100%|██████████| 57/57 [00:05<00:00, 10.94it/s]


...

Epoch 298/300 | Train Loss: 0.0035 | Val Loss: 0.0318 | Val MAPE: 184.52%


Epoch 299/300: 100%|██████████| 57/57 [00:05<00:00, 11.09it/s]


Epoch 299/300 | Train Loss: 0.0054 | Val Loss: 0.0346 | Val MAPE: 65.23%


Epoch 300/300: 100%|██████████| 57/57 [00:05<00:00, 11.07it/s]


Epoch 300/300 | Train Loss: 0.0042 | Val Loss: 0.0421 | Val MAPE: 162.80%
✅ Submission saved to submission.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tabm import TabM  # Ensure TabM is installed and imported

# ------------------------------ Config ------------------------------

class Config:
    TRAIN_PATH = 'train.csv'
    TEST_PATH = 'test.csv'
    TARGET_COLS = [f'BlendProperty{i}' for i in range(1, 11)]
    TEST_SIZE = 0.1
    BATCH_SIZE = 32
    LR = 1e-4
    EPOCHS = 300
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    K_ENSEMBLE = 64
    RANDOM_STATE = 42

# ------------------------------ Dataset ------------------------------

class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

# ------------------------------ Trainer ------------------------------

class Trainer:
    def __init__(self, config):
        self.config = config
        self.device = config.DEVICE
        self.criterion_mse = nn.MSELoss()
        self.load_and_prepare_data()
        self.create_model()

    def load_and_prepare_data(self):
        train_df = pd.read_csv(self.config.TRAIN_PATH)
        test_df = pd.read_csv(self.config.TEST_PATH)
        train_df = self.feature_engineering(train_df, fit=True)
        test_df = self.feature_engineering(test_df, fit=False)

        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            train_df.drop(columns=self.target_cols).values,
            train_df[self.target_cols].values,
            test_size=self.config.TEST_SIZE,
            random_state=self.config.RANDOM_STATE
        )

        self.X_test = test_df.drop(columns=self.target_cols, errors='ignore').values
        self.test_ids = test_df['ID'].values if 'ID' in test_df.columns else np.arange(len(test_df))

        self.num_scaler = RobustScaler()
        self.y_scaler = RobustScaler()

        self.X_train = self.num_scaler.fit_transform(self.X_train)
        self.X_val = self.num_scaler.transform(self.X_val)
        self.X_test = self.num_scaler.transform(self.X_test)

        self.y_train = self.y_scaler.fit_transform(self.y_train)
        self.y_val = self.y_scaler.transform(self.y_val)

        self.train_loader = DataLoader(TabularDataset(self.X_train, self.y_train), batch_size=self.config.BATCH_SIZE, shuffle=True)
        self.val_loader = DataLoader(TabularDataset(self.X_val, self.y_val), batch_size=self.config.BATCH_SIZE, shuffle=False)
        self.test_loader = DataLoader(TabularDataset(self.X_test), batch_size=self.config.BATCH_SIZE, shuffle=False)

    def feature_engineering(self, df, fit=False):
        df = df.copy()
        
        # Ensure all target columns exist — fill missing with zeros
        for col in Config.TARGET_COLS:
            if col not in df.columns:
                df[col] = 0

        # Drop identifier columns if present (e.g. 'ID', 'BlendID')
        id_cols = ['ID', 'BlendID']
        df.drop(columns=[col for col in id_cols if col in df.columns], inplace=True)

        if fit:
            # Determine feature columns during training
            self.feature_cols = [col for col in df.columns if col not in Config.TARGET_COLS]
            self.target_cols = Config.TARGET_COLS

        # Ensure consistent column ordering
        df = df[self.feature_cols + self.target_cols]

        return df


    def create_model(self):
        self.model = TabM.make(
            n_num_features=self.train_loader.dataset.X.shape[1],
            cat_cardinalities=None,
            d_out=len(self.target_cols),
            k=self.config.K_ENSEMBLE
        ).to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.config.LR)

    def train_epoch(self, epoch):
        self.model.train()
        total_loss = 0
        for X, y in tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.config.EPOCHS}"):
            X, y = X.to(self.device), y.to(self.device)
            preds = self.model(X)
            preds_median = torch.median(preds, dim=1)[0]
            loss = self.criterion_mse(preds_median, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(self.train_loader)

    def validate(self):
        self.model.eval()
        total_loss = 0
        all_preds, all_targets = [], []

        with torch.no_grad():
            for X, y in self.val_loader:
                X, y = X.to(self.device), y.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                loss = self.criterion_mse(preds_median, y)
                total_loss += loss.item()

                all_preds.append(preds_median.cpu().numpy())
                all_targets.append(y.cpu().numpy())

        preds_all = np.vstack(all_preds)
        targets_all = np.vstack(all_targets)

        preds_all_orig = self.y_scaler.inverse_transform(preds_all)
        targets_all_orig = self.y_scaler.inverse_transform(targets_all)
        mape = mean_absolute_percentage_error(targets_all_orig, preds_all_orig) * 100

        return total_loss / len(self.val_loader), mape

    def train(self):
        for epoch in range(self.config.EPOCHS):
            train_loss = self.train_epoch(epoch)
            val_loss, val_mape = self.validate()
            print(f"Epoch {epoch+1}/{self.config.EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAPE: {val_mape:.2f}%")

    def predict(self):
        self.model.eval()
        preds_all = []

        with torch.no_grad():
            for X in self.test_loader:
                X = X.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                preds_all.append(preds_median.cpu().numpy())

        preds_all = np.vstack(preds_all)
        preds_orig = self.y_scaler.inverse_transform(preds_all)

        submission = pd.DataFrame(preds_orig, columns=self.target_cols)
        submission.insert(0, 'ID', self.test_ids)
        submission.to_csv('submission3(2).csv', index=False)
        print("✅ Submission saved to submission.csv")

# ------------------------------ Run ------------------------------

if __name__ == '__main__':
    config = Config()
    trainer = Trainer(config)
    trainer.train()
    trainer.predict()


Epoch 1/300: 100%|██████████| 57/57 [00:05<00:00, 10.97it/s]


Epoch 1/300 | Train Loss: 0.5438 | Val Loss: 0.5266 | Val MAPE: 181.83%


Epoch 2/300: 100%|██████████| 57/57 [00:05<00:00, 10.92it/s]


Epoch 2/300 | Train Loss: 0.5414 | Val Loss: 0.5227 | Val MAPE: 182.69%


Epoch 3/300: 100%|██████████| 57/57 [00:05<00:00, 10.59it/s]


...

Epoch 198/300 | Train Loss: 0.0089 | Val Loss: 0.0556 | Val MAPE: 245.91%


Epoch 199/300: 100%|██████████| 57/57 [00:09<00:00,  5.95it/s]


Epoch 199/300 | Train Loss: 0.0086 | Val Loss: 0.0538 | Val MAPE: 237.95%


Epoch 200/300:  95%|█████████▍| 54/57 [00:08<00:00,  6.15it/s]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tabm import TabM  # Ensure TabM is installed and imported

# ------------------------------ Config ------------------------------

class Config:
    TRAIN_PATH = 'train.csv'
    TEST_PATH = 'test.csv'
    TARGET_COLS = [f'BlendProperty{i}' for i in range(1, 11)]
    TEST_SIZE = 0.1
    BATCH_SIZE = 32
    LR = 1e-5
    EPOCHS = 300
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    K_ENSEMBLE = 64
    RANDOM_STATE = 42

# ------------------------------ Dataset ------------------------------

class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

# ------------------------------ Trainer ------------------------------

class Trainer:
    def __init__(self, config):
        self.config = config
        self.device = config.DEVICE
        self.criterion_mse = nn.MSELoss()
        self.load_and_prepare_data()
        self.create_model()

    def load_and_prepare_data(self):
        train_df = pd.read_csv(self.config.TRAIN_PATH)
        test_df = pd.read_csv(self.config.TEST_PATH)
        train_df = self.feature_engineering(train_df, fit=True)
        test_df = self.feature_engineering(test_df, fit=False)

        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            train_df.drop(columns=self.target_cols).values,
            train_df[self.target_cols].values,
            test_size=self.config.TEST_SIZE,
            random_state=self.config.RANDOM_STATE
        )

        self.X_test = test_df.drop(columns=self.target_cols, errors='ignore').values
        self.test_ids = test_df['ID'].values if 'ID' in test_df.columns else np.arange(len(test_df))

        self.num_scaler = RobustScaler()
        self.y_scaler = RobustScaler()

        self.X_train = self.num_scaler.fit_transform(self.X_train)
        self.X_val = self.num_scaler.transform(self.X_val)
        self.X_test = self.num_scaler.transform(self.X_test)

        self.y_train = self.y_scaler.fit_transform(self.y_train)
        self.y_val = self.y_scaler.transform(self.y_val)

        self.train_loader = DataLoader(TabularDataset(self.X_train, self.y_train), batch_size=self.config.BATCH_SIZE, shuffle=True)
        self.val_loader = DataLoader(TabularDataset(self.X_val, self.y_val), batch_size=self.config.BATCH_SIZE, shuffle=False)
        self.test_loader = DataLoader(TabularDataset(self.X_test), batch_size=self.config.BATCH_SIZE, shuffle=False)

    def feature_engineering(self, df, fit=False):
        df = df.copy()
        
        # Ensure all target columns exist — fill missing with zeros
        for col in Config.TARGET_COLS:
            if col not in df.columns:
                df[col] = 0

        # Drop identifier columns if present (e.g. 'ID', 'BlendID')
        id_cols = ['ID', 'BlendID']
        df.drop(columns=[col for col in id_cols if col in df.columns], inplace=True)

        if fit:
            # Determine feature columns during training
            self.feature_cols = [col for col in df.columns if col not in Config.TARGET_COLS]
            self.target_cols = Config.TARGET_COLS

        # Ensure consistent column ordering
        df = df[self.feature_cols + self.target_cols]

        return df


    def create_model(self):
        self.model = TabM.make(
            n_num_features=self.train_loader.dataset.X.shape[1],
            cat_cardinalities=None,
            d_out=len(self.target_cols),
            k=self.config.K_ENSEMBLE
        ).to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.config.LR)

    def train_epoch(self, epoch):
        self.model.train()
        total_loss = 0
        for X, y in tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.config.EPOCHS}"):
            X, y = X.to(self.device), y.to(self.device)
            preds = self.model(X)
            preds_median = torch.median(preds, dim=1)[0]
            loss = self.criterion_mse(preds_median, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(self.train_loader)

    def validate(self):
        self.model.eval()
        total_loss = 0
        all_preds, all_targets = [], []

        with torch.no_grad():
            for X, y in self.val_loader:
                X, y = X.to(self.device), y.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                loss = self.criterion_mse(preds_median, y)
                total_loss += loss.item()

                all_preds.append(preds_median.cpu().numpy())
                all_targets.append(y.cpu().numpy())

        preds_all = np.vstack(all_preds)
        targets_all = np.vstack(all_targets)

        preds_all_orig = self.y_scaler.inverse_transform(preds_all)
        targets_all_orig = self.y_scaler.inverse_transform(targets_all)
        mape = mean_absolute_percentage_error(targets_all_orig, preds_all_orig) * 100

        return total_loss / len(self.val_loader), mape

    def train(self):
        for epoch in range(self.config.EPOCHS):
            train_loss = self.train_epoch(epoch)
            val_loss, val_mape = self.validate()
            print(f"Epoch {epoch+1}/{self.config.EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAPE: {val_mape:.2f}%")

    def predict(self):
        self.model.eval()
        preds_all = []

        with torch.no_grad():
            for X in self.test_loader:
                X = X.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                preds_all.append(preds_median.cpu().numpy())

        preds_all = np.vstack(preds_all)
        preds_orig = self.y_scaler.inverse_transform(preds_all)

        submission = pd.DataFrame(preds_orig, columns=self.target_cols)
        submission.insert(0, 'ID', self.test_ids)
        submission.to_csv('submission3(3).csv', index=False)
        print("✅ Submission saved to submission.csv")

# ------------------------------ Run ------------------------------

if __name__ == '__main__':
    config = Config()
    trainer = Trainer(config)
    trainer.train()
    trainer.predict()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tabm import TabM  # Ensure TabM is installed and imported

# ------------------------------ Config ------------------------------

class Config:
    TRAIN_PATH = 'train.csv'
    TEST_PATH = 'test.csv'
    TARGET_COLS = [f'BlendProperty{i}' for i in range(1, 11)]
    TEST_SIZE = 0.1
    BATCH_SIZE = 32
    LR = 1e-3
    EPOCHS = 300
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    K_ENSEMBLE = 100
    RANDOM_STATE = 42

# ------------------------------ Dataset ------------------------------

class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

# ------------------------------ Trainer ------------------------------

class Trainer:
    def __init__(self, config):
        self.config = config
        self.device = config.DEVICE
        self.criterion_mse = nn.MSELoss()
        self.load_and_prepare_data()
        self.create_model()

    def load_and_prepare_data(self):
        train_df = pd.read_csv(self.config.TRAIN_PATH)
        test_df = pd.read_csv(self.config.TEST_PATH)
        train_df = self.feature_engineering(train_df, fit=True)
        test_df = self.feature_engineering(test_df, fit=False)

        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            train_df.drop(columns=self.target_cols).values,
            train_df[self.target_cols].values,
            test_size=self.config.TEST_SIZE,
            random_state=self.config.RANDOM_STATE
        )

        self.X_test = test_df.drop(columns=self.target_cols, errors='ignore').values
        self.test_ids = test_df['ID'].values if 'ID' in test_df.columns else np.arange(len(test_df))

        self.num_scaler = RobustScaler()
        self.y_scaler = RobustScaler()

        self.X_train = self.num_scaler.fit_transform(self.X_train)
        self.X_val = self.num_scaler.transform(self.X_val)
        self.X_test = self.num_scaler.transform(self.X_test)

        self.y_train = self.y_scaler.fit_transform(self.y_train)
        self.y_val = self.y_scaler.transform(self.y_val)

        self.train_loader = DataLoader(TabularDataset(self.X_train, self.y_train), batch_size=self.config.BATCH_SIZE, shuffle=True)
        self.val_loader = DataLoader(TabularDataset(self.X_val, self.y_val), batch_size=self.config.BATCH_SIZE, shuffle=False)
        self.test_loader = DataLoader(TabularDataset(self.X_test), batch_size=self.config.BATCH_SIZE, shuffle=False)

    def feature_engineering(self, df, fit=False):
        df = df.copy()
        
        # Ensure all target columns exist — fill missing with zeros
        for col in Config.TARGET_COLS:
            if col not in df.columns:
                df[col] = 0

        # Drop identifier columns if present (e.g. 'ID', 'BlendID')
        id_cols = ['ID', 'BlendID']
        df.drop(columns=[col for col in id_cols if col in df.columns], inplace=True)

        if fit:
            # Determine feature columns during training
            self.feature_cols = [col for col in df.columns if col not in Config.TARGET_COLS]
            self.target_cols = Config.TARGET_COLS

        # Ensure consistent column ordering
        df = df[self.feature_cols + self.target_cols]

        return df


    def create_model(self):
        self.model = TabM.make(
            n_num_features=self.train_loader.dataset.X.shape[1],
            cat_cardinalities=None,
            d_out=len(self.target_cols),
            k=self.config.K_ENSEMBLE
        ).to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.config.LR)

    def train_epoch(self, epoch):
        self.model.train()
        total_loss = 0
        for X, y in tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.config.EPOCHS}"):
            X, y = X.to(self.device), y.to(self.device)
            preds = self.model(X)
            preds_median = torch.median(preds, dim=1)[0]
            loss = self.criterion_mse(preds_median, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(self.train_loader)

    def validate(self):
        self.model.eval()
        total_loss = 0
        all_preds, all_targets = [], []

        with torch.no_grad():
            for X, y in self.val_loader:
                X, y = X.to(self.device), y.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                loss = self.criterion_mse(preds_median, y)
                total_loss += loss.item()

                all_preds.append(preds_median.cpu().numpy())
                all_targets.append(y.cpu().numpy())

        preds_all = np.vstack(all_preds)
        targets_all = np.vstack(all_targets)

        preds_all_orig = self.y_scaler.inverse_transform(preds_all)
        targets_all_orig = self.y_scaler.inverse_transform(targets_all)
        mape = mean_absolute_percentage_error(targets_all_orig, preds_all_orig) * 100

        return total_loss / len(self.val_loader), mape

    def train(self):
        for epoch in range(self.config.EPOCHS):
            train_loss = self.train_epoch(epoch)
            val_loss, val_mape = self.validate()
            print(f"Epoch {epoch+1}/{self.config.EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAPE: {val_mape:.2f}%")

    def predict(self):
        self.model.eval()
        preds_all = []

        with torch.no_grad():
            for X in self.test_loader:
                X = X.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                preds_all.append(preds_median.cpu().numpy())

        preds_all = np.vstack(preds_all)
        preds_orig = self.y_scaler.inverse_transform(preds_all)

        submission = pd.DataFrame(preds_orig, columns=self.target_cols)
        submission.insert(0, 'ID', self.test_ids)
        submission.to_csv('submission3(4).csv', index=False)
        print("✅ Submission saved to submission.csv")

# ------------------------------ Run ------------------------------

if __name__ == '__main__':
    config = Config()
    trainer = Trainer(config)
    trainer.train()
    trainer.predict()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tabm import TabM  # Ensure TabM is installed and imported

# ------------------------------ Config ------------------------------

class Config:
    TRAIN_PATH = 'train.csv'
    TEST_PATH = 'test.csv'
    TARGET_COLS = [f'BlendProperty{i}' for i in range(1, 11)]
    TEST_SIZE = 0.1
    BATCH_SIZE = 32
    LR = 1e-3
    EPOCHS = 300
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    K_ENSEMBLE = 200
    RANDOM_STATE = 42

# ------------------------------ Dataset ------------------------------

class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

# ------------------------------ Trainer ------------------------------

class Trainer:
    def __init__(self, config):
        self.config = config
        self.device = config.DEVICE
        self.criterion_mse = nn.MSELoss()
        self.load_and_prepare_data()
        self.create_model()

    def load_and_prepare_data(self):
        train_df = pd.read_csv(self.config.TRAIN_PATH)
        test_df = pd.read_csv(self.config.TEST_PATH)
        train_df = self.feature_engineering(train_df, fit=True)
        test_df = self.feature_engineering(test_df, fit=False)

        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            train_df.drop(columns=self.target_cols).values,
            train_df[self.target_cols].values,
            test_size=self.config.TEST_SIZE,
            random_state=self.config.RANDOM_STATE
        )

        self.X_test = test_df.drop(columns=self.target_cols, errors='ignore').values
        self.test_ids = test_df['ID'].values if 'ID' in test_df.columns else np.arange(len(test_df))

        self.num_scaler = RobustScaler()
        self.y_scaler = RobustScaler()

        self.X_train = self.num_scaler.fit_transform(self.X_train)
        self.X_val = self.num_scaler.transform(self.X_val)
        self.X_test = self.num_scaler.transform(self.X_test)

        self.y_train = self.y_scaler.fit_transform(self.y_train)
        self.y_val = self.y_scaler.transform(self.y_val)

        self.train_loader = DataLoader(TabularDataset(self.X_train, self.y_train), batch_size=self.config.BATCH_SIZE, shuffle=True)
        self.val_loader = DataLoader(TabularDataset(self.X_val, self.y_val), batch_size=self.config.BATCH_SIZE, shuffle=False)
        self.test_loader = DataLoader(TabularDataset(self.X_test), batch_size=self.config.BATCH_SIZE, shuffle=False)

    def feature_engineering(self, df, fit=False):
        df = df.copy()
        
        # Ensure all target columns exist — fill missing with zeros
        for col in Config.TARGET_COLS:
            if col not in df.columns:
                df[col] = 0

        # Drop identifier columns if present (e.g. 'ID', 'BlendID')
        id_cols = ['ID', 'BlendID']
        df.drop(columns=[col for col in id_cols if col in df.columns], inplace=True)

        if fit:
            # Determine feature columns during training
            self.feature_cols = [col for col in df.columns if col not in Config.TARGET_COLS]
            self.target_cols = Config.TARGET_COLS

        # Ensure consistent column ordering
        df = df[self.feature_cols + self.target_cols]

        return df


    def create_model(self):
        self.model = TabM.make(
            n_num_features=self.train_loader.dataset.X.shape[1],
            cat_cardinalities=None,
            d_out=len(self.target_cols),
            k=self.config.K_ENSEMBLE
        ).to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.config.LR)

    def train_epoch(self, epoch):
        self.model.train()
        total_loss = 0
        for X, y in tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.config.EPOCHS}"):
            X, y = X.to(self.device), y.to(self.device)
            preds = self.model(X)
            preds_median = torch.median(preds, dim=1)[0]
            loss = self.criterion_mse(preds_median, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(self.train_loader)

    def validate(self):
        self.model.eval()
        total_loss = 0
        all_preds, all_targets = [], []

        with torch.no_grad():
            for X, y in self.val_loader:
                X, y = X.to(self.device), y.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                loss = self.criterion_mse(preds_median, y)
                total_loss += loss.item()

                all_preds.append(preds_median.cpu().numpy())
                all_targets.append(y.cpu().numpy())

        preds_all = np.vstack(all_preds)
        targets_all = np.vstack(all_targets)

        preds_all_orig = self.y_scaler.inverse_transform(preds_all)
        targets_all_orig = self.y_scaler.inverse_transform(targets_all)
        mape = mean_absolute_percentage_error(targets_all_orig, preds_all_orig) * 100

        return total_loss / len(self.val_loader), mape

    def train(self):
        for epoch in range(self.config.EPOCHS):
            train_loss = self.train_epoch(epoch)
            val_loss, val_mape = self.validate()
            print(f"Epoch {epoch+1}/{self.config.EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAPE: {val_mape:.2f}%")

    def predict(self):
        self.model.eval()
        preds_all = []

        with torch.no_grad():
            for X in self.test_loader:
                X = X.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                preds_all.append(preds_median.cpu().numpy())

        preds_all = np.vstack(preds_all)
        preds_orig = self.y_scaler.inverse_transform(preds_all)

        submission = pd.DataFrame(preds_orig, columns=self.target_cols)
        submission.insert(0, 'ID', self.test_ids)
        submission.to_csv('submission3(5).csv', index=False)
        print("✅ Submission saved to submission.csv")

# ------------------------------ Run ------------------------------

if __name__ == '__main__':
    config = Config()
    trainer = Trainer(config)
    trainer.train()
    trainer.predict()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_percentage_error
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tabm import TabM  # Ensure TabM is installed and imported

# ------------------------------ Config ------------------------------

class Config:
    TRAIN_PATH = 'train.csv'
    TEST_PATH = 'test.csv'
    TARGET_COLS = [f'BlendProperty{i}' for i in range(1, 11)]
    TEST_SIZE = 0.1
    BATCH_SIZE = 32
    LR = 1e-4
    EPOCHS = 300
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    K_ENSEMBLE = 200
    RANDOM_STATE = 42

# ------------------------------ Dataset ------------------------------

class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

# ------------------------------ Trainer ------------------------------

class Trainer:
    def __init__(self, config):
        self.config = config
        self.device = config.DEVICE
        self.criterion_mse = nn.MSELoss()
        self.load_and_prepare_data()
        self.create_model()

    def load_and_prepare_data(self):
        train_df = pd.read_csv(self.config.TRAIN_PATH)
        test_df = pd.read_csv(self.config.TEST_PATH)
        train_df = self.feature_engineering(train_df, fit=True)
        test_df = self.feature_engineering(test_df, fit=False)

        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            train_df.drop(columns=self.target_cols).values,
            train_df[self.target_cols].values,
            test_size=self.config.TEST_SIZE,
            random_state=self.config.RANDOM_STATE
        )

        self.X_test = test_df.drop(columns=self.target_cols, errors='ignore').values
        self.test_ids = test_df['ID'].values if 'ID' in test_df.columns else np.arange(len(test_df))

        self.num_scaler = RobustScaler()
        self.y_scaler = RobustScaler()

        self.X_train = self.num_scaler.fit_transform(self.X_train)
        self.X_val = self.num_scaler.transform(self.X_val)
        self.X_test = self.num_scaler.transform(self.X_test)

        self.y_train = self.y_scaler.fit_transform(self.y_train)
        self.y_val = self.y_scaler.transform(self.y_val)

        self.train_loader = DataLoader(TabularDataset(self.X_train, self.y_train), batch_size=self.config.BATCH_SIZE, shuffle=True)
        self.val_loader = DataLoader(TabularDataset(self.X_val, self.y_val), batch_size=self.config.BATCH_SIZE, shuffle=False)
        self.test_loader = DataLoader(TabularDataset(self.X_test), batch_size=self.config.BATCH_SIZE, shuffle=False)

    def feature_engineering(self, df, fit=False):
        df = df.copy()
        
        # Ensure all target columns exist — fill missing with zeros
        for col in Config.TARGET_COLS:
            if col not in df.columns:
                df[col] = 0

        # Drop identifier columns if present (e.g. 'ID', 'BlendID')
        id_cols = ['ID', 'BlendID']
        df.drop(columns=[col for col in id_cols if col in df.columns], inplace=True)

        if fit:
            # Determine feature columns during training
            self.feature_cols = [col for col in df.columns if col not in Config.TARGET_COLS]
            self.target_cols = Config.TARGET_COLS

        # Ensure consistent column ordering
        df = df[self.feature_cols + self.target_cols]

        return df


    def create_model(self):
        self.model = TabM.make(
            n_num_features=self.train_loader.dataset.X.shape[1],
            cat_cardinalities=None,
            d_out=len(self.target_cols),
            k=self.config.K_ENSEMBLE
        ).to(self.device)

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.config.LR)

    def train_epoch(self, epoch):
        self.model.train()
        total_loss = 0
        for X, y in tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{self.config.EPOCHS}"):
            X, y = X.to(self.device), y.to(self.device)
            preds = self.model(X)
            preds_median = torch.median(preds, dim=1)[0]
            loss = self.criterion_mse(preds_median, y)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()
        return total_loss / len(self.train_loader)

    def validate(self):
        self.model.eval()
        total_loss = 0
        all_preds, all_targets = [], []

        with torch.no_grad():
            for X, y in self.val_loader:
                X, y = X.to(self.device), y.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                loss = self.criterion_mse(preds_median, y)
                total_loss += loss.item()

                all_preds.append(preds_median.cpu().numpy())
                all_targets.append(y.cpu().numpy())

        preds_all = np.vstack(all_preds)
        targets_all = np.vstack(all_targets)

        preds_all_orig = self.y_scaler.inverse_transform(preds_all)
        targets_all_orig = self.y_scaler.inverse_transform(targets_all)
        mape = mean_absolute_percentage_error(targets_all_orig, preds_all_orig) * 100

        return total_loss / len(self.val_loader), mape

    def train(self):
        for epoch in range(self.config.EPOCHS):
            train_loss = self.train_epoch(epoch)
            val_loss, val_mape = self.validate()
            print(f"Epoch {epoch+1}/{self.config.EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAPE: {val_mape:.2f}%")

    def predict(self):
        self.model.eval()
        preds_all = []

        with torch.no_grad():
            for X in self.test_loader:
                X = X.to(self.device)
                preds = self.model(X)
                preds_median = torch.median(preds, dim=1)[0]
                preds_all.append(preds_median.cpu().numpy())

        preds_all = np.vstack(preds_all)
        preds_orig = self.y_scaler.inverse_transform(preds_all)

        submission = pd.DataFrame(preds_orig, columns=self.target_cols)
        submission.insert(0, 'ID', self.test_ids)
        submission.to_csv('submission3(6).csv', index=False)
        print("✅ Submission saved to submission.csv")

# ------------------------------ Run ------------------------------

if __name__ == '__main__':
    config = Config()
    trainer = Trainer(config)
    trainer.train()
    trainer.predict()
